Based on Estimating local and global feature importance scores using DiCE:
http://interpret.ml/DiCE/notebooks/DiCE_feature_importances.html#Local-feature-importance

In [2]:
import os

In [3]:
os.environ["XLA_FLAGS"] = '--xla_gpu_cuda_data_dir=/usr/lib/cuda'

In [6]:
import pathlib

import pandas as pd
import dice_ml
from dice_ml import Dice

from best_models import retrieve_models_by_id
from check_and_prepare_dataset import load_dataset
from manage_model import mm_load_model
from manage_model import allowgrowthgpus

In [7]:
allowgrowthgpus()

In [8]:
data_dir = "/mnt/nvme2tb/ffp/datasets/"
models_dir = "/mnt/nvme2tb/ffp/results/bestmodels"

filters = ["df_flt['params'].str.contains(\"'dropout': None\")"]
opt_targets = [
    "auc", "f1-score 1", "hybrid1", "hybrid2", "hybrid5", "NH2", "NH5", "NH10"
]
testfpattern = cvrespattern = "*NN_ns*mean*"
testmodel = retrieve_models_by_id(785, "hybrid2 test", models_dir, testfpattern,
                                  opt_targets, "val.", "test", filters, 3)

x_train, y_train, _ = load_dataset(pathlib.Path(data_dir,
                                                "train/train_new_sample_1_2_norm.csv"),
                             testmodel["params"]["feature_drop"])
x_train = x_train.reindex(sorted(x_train.columns), axis=1)

Loading full dataset /mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_norm.csv
before nan drop: 41771
after nan drop: 41771
after dup. drop: 41684
renaming "x": "xpos", "y": "ypos"
Ignored columns from csv ['index', 'y', 'x', 'dom_dir', 'dir_max', 'fire', 'weekday', 'month', 'band', 'firedate']
Dropped columns ['bin_corine_gr33', 'bin_month_5', 'bin_month_4', 'bin_corine_gr32', 'bin_dir_max_4', 'bin_weekday_1', 'bin_corine_gr5', 'bin_dir_max_3', 'bin_corine_gr23', 'bin_corine_gr24', 'bin_weekday_6', 'bin_month_7', 'bin_dom_dir_7', 'bin_month_9', 'bin_dir_max_2', 'bin_weekday_4', 'bin_corine_gr31', 'bin_weekday_7', 'bin_month_8', 'bin_corine_gr1', 'bin_corine_gr22', 'bin_dom_dir_4', 'bin_dir_max_6', 'bin_month_3', 'bin_corine_gr4', 'bin_dom_dir_6', 'pop', 'bin_month_6', 'bin_corine_gr21', 'bin_dir_max_1', 'bin_dom_dir_3', 'bin_dom_dir_2', 'bin_weekday_2', 'bin_month_10', 'bin_dom_dir_1', 'bin_dom_dir_8', 'bin_dir_max_5', 'bin_weekday_3', 'bin_dom_dir_5', 'bin_dir_max_8', 'bin_dir_ma

In [9]:
x_test, y_test, _ = load_dataset(pathlib.Path(
    data_dir, "test/2019/20190901_df_norm.csv"),
                                 testmodel["params"]["feature_drop"])
x_test = x_test.reindex(sorted(x_test.columns), axis=1)

Loading full dataset /mnt/nvme2tb/ffp/datasets/test/2019/20190901_df_norm.csv
before nan drop: 656089
after nan drop: 656089
after dup. drop: 656089
renaming "x": "xpos", "y": "ypos"
Ignored columns from csv ['index', 'y', 'x', 'dom_dir', 'dir_max', 'fire', 'weekday', 'month', 'band', 'id', 'firedate']
Dropped columns ['bin_corine_gr33', 'bin_month_5', 'bin_month_4', 'bin_corine_gr32', 'bin_dir_max_4', 'bin_weekday_1', 'bin_corine_gr5', 'bin_dir_max_3', 'bin_corine_gr23', 'bin_corine_gr24', 'bin_weekday_6', 'bin_month_7', 'bin_dom_dir_7', 'bin_month_9', 'bin_dir_max_2', 'bin_weekday_4', 'bin_corine_gr31', 'bin_weekday_7', 'bin_month_8', 'bin_corine_gr1', 'bin_corine_gr22', 'bin_dom_dir_4', 'bin_dir_max_6', 'bin_month_3', 'bin_corine_gr4', 'bin_dom_dir_6', 'pop', 'bin_month_6', 'bin_corine_gr21', 'bin_dir_max_1', 'bin_dom_dir_3', 'bin_dom_dir_2', 'bin_weekday_2', 'bin_month_10', 'bin_dom_dir_1', 'bin_dom_dir_8', 'bin_dir_max_5', 'bin_weekday_3', 'bin_dom_dir_5', 'bin_dir_max_8', 'bin_di

In [10]:
dataset = pd.concat([pd.concat([x_train, y_train], axis=1),
                     pd.concat([x_test, y_test], axis=1)])
dataset = dataset.sample(frac=1).reset_index(drop=True)
dataset

,aspect,bin_corine_1,bin_corine_10,bin_corine_11,bin_corine_12,bin_corine_13,bin_corine_14,bin_corine_15,bin_corine_16,bin_corine_17,...,min_dew_temp,min_temp,ndvi,rain_7_days,res_max,road_dens,slope,xpos,ypos,fire
0,0.298039,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.693853,0.780320,0.850083,0.000501,0.127342,0.266662,0.944751,0.407770,0.798094,0.0
1,0.258824,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.605728,0.681913,0.703167,0.001553,0.092148,0.084636,0.950276,0.264065,0.557071,0.0
2,0.819608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.575106,0.652804,0.727500,0.000349,0.072683,0.000000,0.917127,0.408269,0.945467,0.0
3,0.462745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.599021,0.724469,0.600250,0.000992,0.137338,0.037781,0.955801,0.244605,0.721141,0.0
4,0.560784,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.682239,0.821646,0.503750,0.000030,0.212798,0.083644,1.000000,0.360866,0.445272,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
697768,0.149020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.634935,0.742943,0.607917,0.000139,0.076832,0.128074,0.933702,0.289014,0.467777,0.0
697769,0.117647,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.624860,0.683753,0.763500,0.000119,0.077003,0.000000,0.928177,0.229636,0.628943,0.0
697770,0.988235,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.622429,0.782724,0.480000,0.000030,0.128021,0.119114,1.000000,0.361365,0.878677,0.0
697771,0.643137,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.600965,0.749103,0.571750,0.000813,0.112314,0.045047,0.972376,0.240114,0.747276,0.0


In [11]:
model_file = "hypres_tf_ns_ncv_do_2019_model_id_785_r_0_hybrid2test_1.h5"
weights_file = "hypres_tf_ns_ncv_do_2019_weights_id_785_r_0_hybrid2test_1.cpkt"

model = mm_load_model(pathlib.Path(models_dir, "entiremodels", model_file),
                      "tf", testmodel["params"], x_train)
model.load_weights(pathlib.Path(models_dir, "weights", weights_file))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1300)              83200     
                                                                 
 dense_1 (Dense)             (None, 2)                 2602      
                                                                 
Total params: 85,802
Trainable params: 85,802
Non-trainable params: 0
_________________________________________________________________


2023-07-25 15:04:26.675712: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [12]:
d = dice_ml.Data(dataframe=dataset,
                 continuous_features=list(dataset.columns)[:-1],
                 outcome_name="fire")
m = dice_ml.Model(model=model, backend="TF2")

Local feature importance - generate counterfactuals for a given input point.

In [13]:
exp = Dice(d, m, method="random")
query_instance = x_test[1:2]
e1 = exp.generate_counterfactuals(query_instance, total_CFs=10, desired_range=None,
                                  desired_class="opposite",
                                  permitted_range=None, features_to_vary="all")
e1.visualize_as_dataframe(show_only_changes=True)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.41s/it]

Query instance (original outcome : 0)


,aspect,bin_corine_1,bin_corine_10,bin_corine_11,bin_corine_12,bin_corine_13,bin_corine_14,bin_corine_15,bin_corine_16,bin_corine_17,...,min_dew_temp,min_temp,ndvi,rain_7_days,res_max,road_dens,slope,xpos,ypos,fire
0,0.290196,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.530822,0.662114,0.646583,0.004122,0.080139,0.0,0.98895,0.338412,0.983944,0



Diverse Counterfactual set (new outcome: 1.0)


,aspect,bin_corine_1,bin_corine_10,bin_corine_11,bin_corine_12,bin_corine_13,bin_corine_14,bin_corine_15,bin_corine_16,bin_corine_17,...,min_dew_temp,min_temp,ndvi,rain_7_days,res_max,road_dens,slope,xpos,ypos,fire
0,0.2901960784313726,-,-,-,-,-,-,-,-,-,...,0.5308215082908163,0.6621140700120193,0.6465833333333333,0.0041215963816891,0.0801387854984828,-,0.988950276243094,0.3384124593374689,0.983943498638314,1.0
1,0.2901960784313726,-,-,-,-,-,-,-,-,-,...,0.5308215082908163,0.6621140700120193,0.6465833333333333,0.0041215963816891,0.0801387854984828,-,0.988950276243094,0.3384124593374689,0.983943498638314,1.0
2,0.2901960784313726,-,-,-,-,-,-,-,-,-,...,0.5308215082908163,0.6621140700120193,0.6465833333333333,0.0041215963816891,0.0801387854984828,-,0.988950276243094,0.3384124593374689,0.983943498638314,1.0
3,0.2901960784313726,-,-,-,-,-,-,-,-,-,...,0.5308215082908163,0.6621140700120193,0.6465833333333333,0.0041215963816891,0.0801387854984828,-,0.988950276243094,0.3384124593374689,0.983943498638314,1.0
4,0.2901960784313726,-,-,-,-,-,-,-,-,-,...,0.5308215082908163,0.6621140700120193,0.6465833333333333,0.0041215963816891,0.0801387854984828,-,0.988950276243094,0.3384124593374689,0.983943498638314,1.0
5,0.2901960784313726,-,-,-,-,-,-,-,-,-,...,0.5308215082908163,0.6621140700120193,0.6465833333333333,0.0041215963816891,0.0801387854984828,-,0.988950276243094,0.3384124593374689,0.983943498638314,1.0
6,0.2901960784313726,-,-,-,-,-,-,-,-,-,...,0.5308215082908163,0.6621140700120193,0.6465833333333333,0.0041215963816891,0.0801387854984828,-,0.988950276243094,0.3384124593374689,0.983943498638314,1.0
7,0.2901960784313726,-,-,-,-,-,-,-,-,-,...,0.5308215082908163,0.6621140700120193,0.6465833333333333,0.0041215963816891,0.0801387854984828,-,0.988950276243094,0.3384124593374689,0.983943498638314,1.0
8,0.2901960784313726,-,-,-,-,-,-,-,-,-,...,0.5308215082908163,0.6621140700120193,0.6465833333333333,0.0041215963816891,0.0801387854984828,-,0.988950276243094,0.3384124593374689,0.983943498638314,1.0
9,0.2901960784313726,-,-,-,-,-,-,-,-,-,...,0.5308215082908163,0.6621140700120193,0.6465833333333333,0.0041215963816891,0.0801387854984828,-,0.988950276243094,0.3384124593374689,0.983943498638314,1.0


These can now be used to calculate the feature importance scores.

In [14]:
imp = exp.local_feature_importance(query_instance,
                                   cf_examples_list=e1.cf_examples_list)
print(imp.local_importance)

[{'bin_corine_27': 0.4, 'bin_corine_26': 0.1, 'bin_corine_28': 0.1, 'bin_corine_3': 0.1, 'bin_corine_32': 0.1, 'bin_corine_35': 0.1, 'bin_corine_37': 0.1, 'bin_corine_40': 0.1, 'bin_corine_42': 0.1, 'bin_corine_44': 0.1, 'f81': 0.1, 'frequency': 0.1, 'mean_temp': 0.1, 'aspect': 0.0, 'bin_corine_1': 0.0, 'bin_corine_10': 0.0, 'bin_corine_11': 0.0, 'bin_corine_12': 0.0, 'bin_corine_13': 0.0, 'bin_corine_14': 0.0, 'bin_corine_15': 0.0, 'bin_corine_16': 0.0, 'bin_corine_17': 0.0, 'bin_corine_18': 0.0, 'bin_corine_19': 0.0, 'bin_corine_2': 0.0, 'bin_corine_20': 0.0, 'bin_corine_21': 0.0, 'bin_corine_22': 0.0, 'bin_corine_23': 0.0, 'bin_corine_24': 0.0, 'bin_corine_25': 0.0, 'bin_corine_29': 0.0, 'bin_corine_30': 0.0, 'bin_corine_31': 0.0, 'bin_corine_33': 0.0, 'bin_corine_36': 0.0, 'bin_corine_38': 0.0, 'bin_corine_4': 0.0, 'bin_corine_41': 0.0, 'bin_corine_43': 0.0, 'bin_corine_5': 0.0, 'bin_corine_6': 0.0, 'bin_corine_7': 0.0, 'bin_corine_8': 0.0, 'bin_corine_9': 0.0, 'dem': 0.0, 'dom_vel

Feature importance can also be estimated directly, by leaving the cf_examples_list argument blank.

In [15]:
imp = exp.local_feature_importance(query_instance, posthoc_sparsity_param=None)
print(imp.local_importance)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.12it/s]

[{'bin_corine_24': 0.2, 'bin_corine_27': 0.2, 'bin_corine_14': 0.1, 'bin_corine_26': 0.1, 'bin_corine_28': 0.1, 'bin_corine_29': 0.1, 'bin_corine_31': 0.1, 'bin_corine_32': 0.1, 'bin_corine_33': 0.1, 'bin_corine_4': 0.1, 'bin_corine_40': 0.1, 'bin_corine_42': 0.1, 'frequency': 0.1, 'min_dew_temp': 0.1, 'res_max': 0.1, 'aspect': 0.0, 'bin_corine_1': 0.0, 'bin_corine_10': 0.0, 'bin_corine_11': 0.0, 'bin_corine_12': 0.0, 'bin_corine_13': 0.0, 'bin_corine_15': 0.0, 'bin_corine_16': 0.0, 'bin_corine_17': 0.0, 'bin_corine_18': 0.0, 'bin_corine_19': 0.0, 'bin_corine_2': 0.0, 'bin_corine_20': 0.0, 'bin_corine_21': 0.0, 'bin_corine_22': 0.0, 'bin_corine_23': 0.0, 'bin_corine_25': 0.0, 'bin_corine_3': 0.0, 'bin_corine_30': 0.0, 'bin_corine_35': 0.0, 'bin_corine_36': 0.0, 'bin_corine_37': 0.0, 'bin_corine_38': 0.0, 'bin_corine_41': 0.0, 'bin_corine_43': 0.0, 'bin_corine_44': 0.0, 'bin_corine_5': 0.0, 'bin_corine_6': 0.0, 'bin_corine_7': 0.0, 'bin_corine_8': 0.0, 'bin_corine_9': 0.0, 'dem': 0.0, '

Global importance - For global importance, we need to generate counterfactuals for a representative sample of the dataset.

In [16]:
x_test_ = x_test.sample(1000).reset_index(drop=True)

cobj = exp.global_feature_importance(x_test_, total_CFs=10,
                                     posthoc_sparsity_param=None)
print(cobj.summary_importance)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [07:47<00:00,  2.14it/s]


{'bin_corine_33': 0.1036, 'bin_corine_27': 0.0756, 'res_max': 0.0626, 'bin_corine_26': 0.054, 'bin_corine_32': 0.0482, 'bin_corine_42': 0.0442, 'bin_corine_28': 0.0432, 'rain_7_days': 0.0401, 'bin_corine_25': 0.0376, 'f81': 0.0367, 'bin_corine_6': 0.0362, 'bin_corine_20': 0.0353, 'bin_corine_10': 0.0341, 'bin_corine_35': 0.034, 'bin_corine_14': 0.0329, 'bin_corine_18': 0.0326, 'bin_corine_21': 0.0321, 'bin_corine_44': 0.0319, 'bin_corine_29': 0.0317, 'bin_corine_31': 0.0312, 'bin_corine_43': 0.0309, 'dem': 0.0292, 'bin_corine_37': 0.029, 'bin_corine_24': 0.0287, 'bin_corine_8': 0.0267, 'bin_corine_23': 0.0264, 'bin_corine_17': 0.0263, 'bin_corine_38': 0.0258, 'bin_corine_7': 0.0258, 'evi': 0.0252, 'frequency': 0.0247, 'bin_corine_15': 0.0245, 'bin_corine_2': 0.0239, 'bin_corine_1': 0.0222, 'bin_corine_12': 0.0216, 'ypos': 0.0209, 'bin_corine_40': 0.0207, 'max_temp': 0.0207, 'min_dew_temp': 0.0197, 'bin_corine_16': 0.0196, 'dom_vel': 0.0192, 'bin_corine_9': 0.0186, 'mean_temp': 0.0181, 